# Uniref mappings



In [1]:
import org.apache.spark._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [2]:
val whalePath = "/data/results/gray-whale/"
val expressionsPath = whalePath + "Expressions/"
val unirefPath = expressionsPath + "uniref90/"
val transcriptsPath = expressionsPath + "Transcripts/"
val codingPath = transcriptsPath + "coding/"

val comparisonsPath = expressionsPath + "Comparisons/"
val comparisonsUniref = comparisonsPath + "uniref90_comparisons/"
val annotationsPath = comparisonsPath + "annotations/"

In [3]:
def loadTranscripts(subpath: String, prefix: String) = {
    val path = if(subpath.startsWith("/")) subpath else transcriptsPath + subpath
    spark.readTSV(path, header=true).select($"Name".as("transcript"), $"NumReads".as(prefix + "_reads"), $"TPM".as(prefix + "_TPM")).cache 
}

Loading transcripts<br>

In [5]:
val gray_liver_tr = loadTranscripts("raw/gray_whale/liver/transcripts_quant", "gray_whale_liver")
val gray_kidney_tr =  loadTranscripts("raw/gray_whale/kidney/transcripts_quant", "gray_whale_kidney")
(gray_liver_tr.count, gray_kidney_tr.count)

(114263,114263)

In [6]:
val anno = spark.readTSV(comparisonsPath + "annotations/annotations_uniref.tsv", header = true)
anno.limit(10).show(10,1000)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------

## Opening Uniref90 mapping file<br>




In [8]:
val mapping_cols = Seq("UniProtKB-AC","UniProtKB-ID","Entrez","RefSeq","GI","PDB","GO",
"UniRef100","UniRef90","UniRef50","UniParc","PIR",
"NCBI-taxon","MIM","UniGene","PubMed",
"EMBL","EMBL-CDS","Ensembl","Ensembl_TRS","Ensembl_PRO","Additional PubMed")

In [9]:
val mapping = spark.readTSV("/data/indexes/uniprot/idmapping_selected.tab").toDF(mapping_cols:_*)
mapping.limit(20).show(20, 1000)

+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|UniProtKB-AC|UniProtKB-ID| Entrez|     RefSeq|                             GI| PDB|                                GO|       UniRef100|       UniRef90|       UniRef50|      UniParc| PIR|NCBI-taxon| MIM|UniGene|            PubMed|    EMBL|  EMBL-CDS|Ensembl|Ensembl_TRS|Ensembl_PRO|Additional PubMed|
+------------+------------+-------+-----------+-------------------------------+----+----------------------------------+----------------+---------------+---------------+-------------+----+----------+----+-------+------------------+--------+----------+-------+-----------+-----------+-----------------+
|      Q6GZX4|  001R_FRG3G|2947773|YP_031579.1|             81941549; 49237298|null|             

## Processing Bat expressions<br>




In [11]:
val quants_base = "/data/samples/de_novo/bat/quants/ours"
val bat_liver_1 = spark.readTSV(quants_base + "/quant_bat_liver_active_1/quant.sf", header = true)
val bat_kidney_1 = spark.readTSV(quants_base + "/quant_bat_kidney_active_1/quant.sf", header = true)
val bat_liver_2 = spark.readTSV(quants_base + "/quant_bat_liver_active_2/quant.sf", header = true)
val bat_kidney_2 = spark.readTSV(quants_base + "/quant_bat_kidney_active_2/quant.sf", header = true)
bat_liver_1.show(10,10000)

+-----------------------------------------+------+---------------+---------+--------+
|                                     Name|Length|EffectiveLength|      TPM|NumReads|
+-----------------------------------------+------+---------------+---------+--------+
| NODE_1_length_17988_cov_580.222903_g0_i0| 17988|      25996.137| 2.849032|2131.249|
|  NODE_2_length_17947_cov_75.308981_g1_i0| 17947|      23591.296| 5.768584|3916.057|
|  NODE_3_length_17149_cov_48.936455_g2_i0| 17149|      16480.623| 2.802971|1329.291|
| NODE_4_length_16953_cov_143.141987_g3_i0| 16953|      16648.804|11.308167|5417.553|
|  NODE_5_length_16927_cov_28.452499_g4_i0| 16927|      15432.613| 1.503753| 667.796|
|  NODE_6_length_16768_cov_80.455725_g5_i0| 16768|      22827.061| 0.206226| 135.463|
|  NODE_7_length_16714_cov_42.414072_g6_i0| 16714|       13824.43| 1.240009| 493.287|
|  NODE_8_length_16515_cov_85.093674_g5_i1| 16515|      22310.009| 0.971752| 623.855|
|  NODE_9_length_16383_cov_85.107160_g5_i2| 16383|    

### Protein search auxiliary classes




Computing bat mapping<br>

In [14]:
import org.bdgenomics.adam.rdd.ADAMContext._
val bat_protein = spark.sparkContext.loadFastaProtein("/data/results/gray-whale/transdecoder/transdecoder/miotis_brandis_uniref90/transcripts.fasta.transdecoder.pep")
bat_protein.dataset.show(10,10000)

+--------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
val bat_liver_kidney = bat_liver_1.select($"Name".as("transcript"), $"TPM".as("bat_liver_1"))
  .join(bat_kidney_1.select($"Name".as("transcript"), $"TPM".as("bat_kidney_1")), Seq("transcript"))
  .join(bat_liver_2.select($"Name".as("transcript"), $"TPM".as("bat_liver_2")), Seq("transcript"))
  .join(bat_kidney_2.select($"Name".as("transcript"), $"TPM".as("bat_kidney_2")), Seq("transcript"))

bat_liver_kidney.count

408411

In [16]:
def un_protein = udf[String, String](str=> str.substring(0, str.indexOf(".p")))
def to_orf = udf[Array[String], String]{ str => 
 val of = str.substring(str.indexOf("ORF")+4)
 of.substring(0, of.indexOf("NODE")-1).split(',')
}
//def separate = udf[Array[String], String](str => str.split(','))

In [17]:
import scala.collection.mutable.WrappedArray
def get_array(num: Int): UserDefinedFunction = udf[String, WrappedArray[String]](arr => arr(num))
val orf_type = get_array(0)

In [18]:
val orf_score =   udf[Double, WrappedArray[String]](arr => arr(1).replace("score=", "").toDouble)

case class UnirefScore(uniref: String, score: Double, p: String)
def uniref = udf[WrappedArray[UnirefScore], WrappedArray[String]]{case arr => 
    val ars = arr.tail.tail
    ars.map{ case a => 
        val u = a.split('|')
        UnirefScore(u(0), u(1).toDouble, u(2))
    }
    }

def len = udf[Int, String]{ case str=> 
    val l = str.indexOf("len:")
    str.substring(l + 4, str.indexOf(" (")).toInt
}


In [19]:
val bat_protein_dataset_temp = bat_protein.dataset
.withColumn("transcript", un_protein($"name"))
.withColumn("orf",  to_orf($"description"))
.withColumn("orf_type", orf_type($"orf"))
.withColumn("orf_len",  len($"orf_type"))
.withColumn("orf_score", orf_score($"orf"))
.withColumn("uniref", uniref($"orf"))
.select("transcript", "orf_type", "orf_len", "orf_score",  "uniref")
.as[(String, String, Int, Double, scala.collection.mutable.ArrayBuffer[UnirefScore])]
.filter(r=>r._5.size != 0 && r._4 > 0 && r._3 > 100)

val bat_protein_dataset = bat_protein_dataset_temp.join(
    bat_protein_dataset_temp.select("transcript", "orf_score").groupBy("transcript").agg(max("orf_score").as("orf_score")), Seq("transcript", "orf_score")
)
bat_protein_dataset.show(10,10000)

+------------------------------------------------+---------+-------------------------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
(bat_protein_dataset.count,bat_protein_dataset.select("transcript").distinct.count)

(41829,41826)

In [21]:
import org.apache.spark.sql.Row
def best_hit_uniref = udf[String,WrappedArray[Row]]{arr=>
    val row = arr.maxBy(r=>r.getDouble(1))
    row.getString(0)
}
def best_hit_score = udf[Double,WrappedArray[Row]]{arr=>
    val row = arr.maxBy(r=>r.getDouble(1))
    row.getDouble(1)
}

//UniRef90_G1PK73, 93.9 , UniRef90_G1P3P3, 100.0
//UniRef90_UPI000767D85D, 95.6
val best_bat = bat_protein_dataset
.withColumn("best_hit_score", best_hit_score($"uniref"))
.withColumn("uniref90", best_hit_uniref($"uniref"))
.select("uniref90","best_hit_score","transcript", "orf_type", "orf_score")
best_bat.show(10, 1000)


+----------------------+--------------+------------------------------------------------+-------------------------------+---------+
|              uniref90|best_hit_score|                                      transcript|                       orf_type|orf_score|
+----------------------+--------------+------------------------------------------------+-------------------------------+---------+
|       UniRef90_O88632|          96.8|  NODE_10105_length_3408_cov_152.068053_g4858_i0|      type:complete len:749 (-)|   286.54|
|   UniRef90_A0A0N6WYT5|          98.7|   NODE_10176_length_3399_cov_88.702026_g4622_i1|type:5prime_partial len:244 (-)|    72.85|
|   UniRef90_A0A3P4LV69|          99.3|   NODE_103061_length_457_cov_2.806763_g75741_i0|type:3prime_partial len:152 (-)|    59.89|
|UniRef90_UPI0006D70FC8|          90.7|   NODE_10471_length_3352_cov_73.155334_g3776_i4|type:5prime_partial len:200 (-)|    28.15|
|UniRef90_UPI0006D71BA1|         100.0|   NODE_106047_length_445_cov_8.014925_g7871

In [22]:
val bat_mappings = best_bat.join(bat_liver_kidney , Seq("transcript"))
bat_mappings.show(10,1000)

+------------------------------------------------+----------------------+--------------+-------------------------------+---------+-----------+------------+-----------+------------+
|                                      transcript|              uniref90|best_hit_score|                       orf_type|orf_score|bat_liver_1|bat_kidney_1|bat_liver_2|bat_kidney_2|
+------------------------------------------------+----------------------+--------------+-------------------------------+---------+-----------+------------+-----------+------------+
|   NODE_101874_length_462_cov_1.926014_g74561_i0|       UniRef90_S7NS67|         100.0|type:5prime_partial len:135 (+)|    12.29|   0.672502|    0.580052|   0.496259|    0.119216|
|   NODE_10277_length_3384_cov_126.237055_g742_i2|       UniRef90_G1PWG9|          97.2|      type:complete len:575 (+)|    99.66|        0.0|    1.599671|    0.07182|    0.372074|
|   NODE_103361_length_456_cov_2.174334_g76040_i0|       UniRef90_S7NFL5|          72.1|type:5p

In [23]:
(best_bat.count, best_bat.select("uniref90").distinct.count, best_bat.select("transcript").count, best_bat.select("transcript").distinct.count, best_bat.select("transcript", "uniref90").distinct.count)

(41829,25503,41829,41826,41826)

In [24]:
bat_mappings.count()

41829

In [25]:
bat_mappings.writeTSV("/data/results/gray-whale/mappings/bat_mappings.tsv", header = true)
bat_mappings.writeParquet("/data/results/gray-whale/best_bap_mappings.parquet", local = true)
//val bat_mappings = spark.readTSV("/data/results/gray-whale/mappings/bat_mappings.tsv", header = true) //spark.readParquet("/data/results/gray-whale/best_bap_mappings.parquet")
//bat_mappings.show(10,1000)

parts of /data/results/gray-whale/mappings/bat_mappings.tsv merged!
parts of /data/results/gray-whale/best_bap_mappings.parquet merged!


/data/results/gray-whale/best_bap_mappings.parquet

# Summarizing transcripts by Uniref90




In [27]:
val bat_mappings = spark.read.parquet("/data/results/gray-whale/best_bap_mappings.parquet")

In [28]:
import org.apache.spark.sql.functions._
val bat_unirefs = bat_mappings.select($"uniref90".as("UniRef90"),$"bat_liver_1",$"bat_kidney_1",$"bat_liver_2",$"bat_kidney_2")
.groupBy("UniRef90").agg(
    sum($"bat_liver_1").as("bat_liver_1"), 
    sum($"bat_kidney_1").as("bat_kidney_1"), 
    sum($"bat_liver_2").as("bat_liver_2"), 
    sum($"bat_kidney_2").as("bat_kidney_2")
    ).orderBy($"bat_liver_1".desc, $"bat_kidney_1".desc)
bat_unirefs.show(10,1000)

+----------------------+------------------+-----------------+------------------+------------------+
|              UniRef90|       bat_liver_1|     bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+----------------------+------------------+-----------------+------------------+------------------+
|       UniRef90_G1PM73|19697.715009000003|         5.018056|19624.926098999997| 9.537035000000001|
|       UniRef90_S7PWY9|      19688.750414|         5.501245|       15423.90906|          8.739993|
|       UniRef90_S7NKH8|13626.893756000001|         6.533802|14379.784097999996|         11.232051|
|       UniRef90_L5LNJ6|      13496.735069|         0.464551|15308.018013000003|0.6780390000000001|
|       UniRef90_S7MRF2|       8993.596224|         5.323449|       8062.678561|          5.772969|
|UniRef90_UPI000CCC3436|       7970.125527|         4.011104|       8671.107356|          6.352832|
|       UniRef90_S7MHE7|       6585.148337|         1.531405|       5983.654563|          2.328961|


In [29]:
import group.research.aging.spark.extensions.functions.ConcatenateString
val con = new ConcatenateString(";")
val partial_bat_uniref_mappings = mapping.select("UniRef90", "GO").join(bat_unirefs.select("UniRef90"), "UniRef90").distinct.na.fill("")
.groupBy($"UniRef90").agg(con($"GO").as("GO")).cache
partial_bat_uniref_mappings.show(10,1000)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|           UniRef90|                                                                                                                                                                                                                                                                                                                                                                    GO|
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
(partial_bat_uniref_mappings.count, partial_bat_uniref_mappings.select("UniRef90").distinct.count, bat_unirefs.count,bat_unirefs.select("UniRef90").distinct.count)

(18459,18459,25503,25503)

<div><h2>Aggregating GOes by Uniref90<br></h2></div>

In [32]:
def flatGo = udf[Array[String], String]{ str => str.trim.replaceAll("^;", "".replaceAll(" +", "")).split(";").distinct}
val partial_bat_go_mappings = partial_bat_uniref_mappings
.filter($"GO" =!=";")
.withColumn("GO", flatGo($"GO"))
.join(bat_unirefs, Seq("Uniref90"))
.as[(String, scala.collection.mutable.ArrayBuffer[String], Double, Double, Double, Double)]
partial_bat_go_mappings.show(10,1000)

+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+------------------+------------------+------------------+
|           UniRef90|                                                                                                                                                                                                                                                                                                                     GO|bat_liver_1|      bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [39]:
val bat_go_mappings = partial_bat_go_mappings
.flatMap{ case (a, arr, l1, k1, l2, k2) => arr.toList.filter(f=>f.trim!="").distinct.map(aa=> (aa.trim, l1, k1, l2, k2 ))}
.toDF("GO", "bat_liver_1", "bat_kidney_1", "bat_liver_2", "bat_kidney_2")
.groupBy($"GO").agg(sum($"bat_liver_1").as("bat_liver_1"), sum($"bat_kidney_1").as("bat_kidney_1"), sum($"bat_liver_2").as("bat_liver_2"), sum($"bat_kidney_2").as("bat_kidney_2"))
.orderBy($"bat_liver_1".desc,$"bat_kidney_1".desc)
.withColumnRenamed("GO", "go")
bat_go_mappings.show(10,100)

In [40]:
//previos (13694,13694,13694)
(bat_go_mappings.count, bat_go_mappings.select("go").count, bat_go_mappings.select("go").distinct.count)

In [35]:
bat_go_mappings.select("go").groupBy("go").agg(count($"go")).orderBy($"count(go)".desc).show(100)

+----------+---------+
|        go|count(go)|
+----------+---------+
|GO:0016020|        1|
|GO:0014069|        1|
|GO:0007283|        1|
|GO:0051015|        1|
|GO:0006635|        1|
|GO:0005643|        1|
|GO:1990416|        1|
|GO:2000010|        1|
|GO:0010716|        1|
|GO:0032411|        1|
|GO:0000123|        1|
|GO:0001894|        1|
|GO:0046475|        1|
|GO:0010666|        1|
|GO:0006851|        1|
|GO:0010988|        1|
|GO:0050431|        1|
|GO:0007062|        1|
|GO:0051893|        1|
|GO:0071423|        1|
|GO:0050787|        1|
|GO:0036529|        1|
|GO:0018344|        1|
|GO:0052689|        1|
|GO:0016233|        1|
|GO:0008144|        1|
|GO:0086091|        1|
|GO:0030506|        1|
|GO:0008312|        1|
|GO:0008475|        1|
|GO:0045657|        1|
|GO:0004775|        1|
|GO:0030854|        1|
|GO:0085032|        1|
|GO:0004693|        1|
|GO:0030263|        1|
|GO:2000483|        1|
|GO:0046886|        1|
|GO:0072572|        1|
|GO:2000626|        1|
|GO:0043519

In [36]:
bat_go_mappings.writeTSV("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab")
//bat_go_mappings.writeParquet("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.parquet", true)

parts of /data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab merged!


/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab

In [37]:
val batGO = spark.readTSV("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab", true)
//batGO.show(10)
(batGO.count, batGO.select("go").count, batGO.select("go").distinct.count)

(12981,12981,12981)